# Draft Assistant

A code-based draft assistant (no UI).

In [1]:
# Install package
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Show available drafting models
sd.list_sets()

['AFR',
 'BLB',
 'BRO',
 'DMU',
 'DSK',
 'FDN',
 'HBG',
 'KTK',
 'LCI',
 'LTR',
 'MH3',
 'MID',
 'MKM',
 'MOM',
 'NEO',
 'ONE',
 'OTJ',
 'SIR',
 'SNC',
 'STX',
 'VOW',
 'WOE']

In [13]:
# Load a model. 
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [14]:
# Draft assistant
# collection = ["Vivien Reid", "Llanowar Elves", "Refute", "Bigfin Bouncer"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
collection = []
pick_order = dm.get_pick_order(collection)
pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])]
pick_order

# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(collection, starting_colors="")

,name,rarity,color_identity,p1p1_rating,synergy,rating
174,Mischievous Mystic,uncommon,U,3.64,0.0,3.64
267,Vampire Nighthawk,uncommon,B,3.57,0.0,3.57
50,Cat Collector,uncommon,W,3.33,0.0,3.33
244,Sun-Blessed Healer,uncommon,W,3.24,0.0,3.24
101,Fiery Annihilation,uncommon,R,3.13,0.0,3.13
94,Faebloom Trick,uncommon,U,3.08,0.0,3.08
46,Burst Lightning,common,R,3.02,0.0,3.02
86,Empyrean Eagle,uncommon,Multicolor,3.02,0.0,3.02
233,Spectral Sailor,uncommon,U,2.99,0.0,2.99
74,Dreadwing Scavenger,uncommon,Multicolor,2.97,0.0,2.97


# Sealed/Deckbuilding Assistant

In [15]:
# Paste sealed (or draft) pool from Arena export into notebook
# For detailed instructions on exporting a deck, see: https://draftsim.com/mtg-arena-export-deck/
sealed_pool = """2 Healer's Hawk (FDN) 142
1 Mountain (FDN) 279
2 Joust Through (FDN) 19
1 Ajani's Pridemate (FDN) 135
1 Banishing Light (FDN) 138
1 Dazzling Angel (FDN) 9
1 Inspiring Paladin (FDN) 18
1 Armasaur Guide (FDN) 3
1 An Offer You Can't Refuse (FDN) 160
1 Swamp (FDN) 277
1 Witness Protection (FDN) 168
1 Aetherize (FDN) 151
1 Lightshell Duo (FDN) 157
1 Extravagant Replication (FDN) 154
1 Grappling Kraken (FDN) 39
3 Island (FDN) 275
1 Pilfer (FDN) 181
4 Plains (FDN) 273
1 Frenzied Goblin (FDN) 199
1 Kellan, Planar Trailblazer (FDN) 91
1 Rite of the Dragoncaller (FDN) 92
1 Mild-Mannered Librarian (FDN) 228
3 Forest (FDN) 281
1 Genesis Wave (FDN) 221
1 Scoured Barrens (FDN) 266
1 Dismal Backwater (FDN) 261
1 Swiftwater Cliffs (FDN) 268
1 Jungle Hollow (FDN) 263
1 Wardens of the Cycle (FDN) 125
1 Evolving Wilds (FDN) 262
1 Adventuring Gear (FDN) 249

Sideboard
1 Goblin Boarders (FDN) 87
2 Vampire Soulcaller (FDN) 75
1 Gutless Plunderer (FDN) 60
2 Think Twice (FDN) 165
1 Wary Thespian (FDN) 235
1 Serra Angel (FDN) 147
2 Garruk's Uprising (FDN) 220
1 Stromkirk Bloodthief (FDN) 185
1 Spinner of Souls (FDN) 112
2 Axgard Cavalry (FDN) 189
1 Dwynen's Elite (FDN) 218
1 Involuntary Employment (FDN) 203
1 Helpful Hunter (FDN) 16
2 Erudite Wizard (FDN) 37
1 Burnished Hart (FDN) 250
1 Blanchwood Armor (FDN) 213
1 Phyrexian Arena (FDN) 180
1 Inspiring Paladin (FDN) 18
1 Courageous Goblin (FDN) 82
2 Hungry Ghoul (FDN) 62
1 Prideful Parent (FDN) 21
2 Broken Wings (FDN) 214
1 Campus Guide (FDN) 251
1 Tragic Banshee (FDN) 73
1 Tatyova, Benthic Druid (FDN) 247
1 Inspiration from Beyond (FDN) 43
1 Genesis Wave (FDN) 221
1 Treetop Snarespinner (FDN) 114
2 Fanatical Firebrand (FDN) 195
1 Gleaming Barrier (FDN) 252
1 Bite Down (FDN) 212
1 Soul-Shackled Zombie (FDN) 70
1 Uncharted Voyage (FDN) 53
1 Wardens of the Cycle (FDN) 125
1 Slumbering Cerberus (FDN) 94
1 Drakuseth, Maw of Flames (FDN) 193
1 Strix Lookout (FDN) 52
1 Spitfire Lagac (FDN) 208
1 Stab (FDN) 71
1 Apothecary Stomper (FDN) 99
1 Infernal Vessel (FDN) 63
1 Tolarian Terror (FDN) 167
1 Icewind Elemental (FDN) 42
1 Cackling Prowler (FDN) 101
1 Cephalid Inkmage (FDN) 32
1 Youthful Valkyrie (FDN) 149
"""

In [16]:
# Parse into collection list
sealed_cardnames = sd.parse_cardnames(sealed_pool)

# Get deckbuild recommendations - expect to need to tweak a few cards. 
dm = sd.DraftModel(set="FDN")
deck_rec = dm.get_deck_recommendation(sealed_cardnames, starting_colors="BR")
deck_rec

,name,color_identity,rating
17,"Kellan, Planar Trailblazer",R,4.55
71,"Drakuseth, Maw of Flames",R,3.74
39,Spinner of Souls,G,3.70
76,Infernal Vessel,B,3.69
58,Tragic Banshee,B,3.66
74,Stab,B,3.63
49,Phyrexian Arena,B,3.62
53,Hungry Ghoul,B,3.34
52,Hungry Ghoul,B,3.34
70,Slumbering Cerberus,R,3.25
